In [1]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 建立连接:
s.connect(('127.0.0.1', 6666))

print('客户端发送消息：Time')
s.send(b"Time")
print("服务器当前时间为:",s.recv(1024).decode('utf-8'))
print('客户端发送消息：exit')
s.send(b'exit')
print("客户端接收消息：",s.recv(1024).decode('utf-8'))
s.close()

客户端发送消息：Time
服务器当前时间为: Thu, Apr 04 21:34
客户端发送消息：exit
客户端接收消息： Bye


In [ ]:
from tkinter import *
import time
import socket
import threading

sock = socket.socket(type=socket.SOCK_DGRAM)
sock.bind(('127.0.0.1', 8081))
def rec(sock):  
    while True:  
        data, addr = sock.recvfrom(1024)  #接收信息
        msg = 'server'+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())+'\n'#显示发送信息
        txt_msglist.insert(END,msg,'green') #添加时间 
        txt_msglist.insert(END,data.decode('utf-8')+'\n')

'''
定义消息发送函数：
1、在<消息列表分区>的文本控件中实时添加时间；
2、获取<发送消息分区>的文本内容，添加到列表分区的文本中；
3、将<发送消息分区>的文本内容清空。
'''
def msgsend():
    messages = txt_msgsend.get('0.0',END).strip().encode('utf-8')#发送
    sock.sendto(messages, ('127.0.0.1',8080))
    
    msg = 'client'+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())+'\n'
    txt_msglist.insert(END,msg,'green') #添加时间
    txt_msglist.insert(END,txt_msgsend.get('0.0',END)) #获取发送消息，添加文本到消息列表
    txt_msgsend.delete('0.0',END) #清空发送消息    
 
'''定义取消发送 消息 函数'''
def cancel():
    txt_msgsend.delete('0.0',END) #取消发送消息，即清空发送消息
 
'''绑定up键'''
def msgsendEvent(event):
    if event.keysym == 'Up':
        msgsend()

trd=threading.Thread(target=rec,args=(sock,))  
trd.start()  

tk = Tk()

tk.title('客户端聊天窗口')
 
'''创建分区'''
f_msglist = Frame(height = 300,width = 300) #创建<消息列表分区 >  
f_msgsend = Frame(height = 300,width = 300) #创建<发送消息分区 >
f_floor = Frame(height = 100,width = 300)   #创建<按钮分区>
 
'''创建控件'''
txt_msglist = Text(f_msglist) #消息列表分区中创建文本控件
txt_msglist.tag_config('green',foreground = 'blue') #消息列表分区中创建标签
txt_msgsend = Text(f_msgsend) #发送消息分区中创建文本控件
txt_msgsend.bind('<KeyPress-Up>',msgsendEvent) #发送消息分区中，绑定‘UP’键与消息发送。
'''txt_right = Text(f_right) #图片显示分区创建文本控件'''
button_send = Button(f_floor,text = 'Send',command = msgsend) #按钮分区中创建按钮并绑定发送消息函数
button_cancel = Button(f_floor,text = 'Cancel',command = cancel) #分区中创建取消按钮并绑定取消函数
 
'''分区布局'''
f_msglist.grid(row = 0,column = 0 ) #消息列表分区
f_msgsend.grid(row = 1,column = 0)  #发送消息分区
f_floor.grid(row = 2,column = 0)    #按钮分区

txt_msglist.grid()  #消息列表文本控件加载
txt_msgsend.grid()  #消息发送文本控件加载
button_send.grid(row = 0,column = 0,sticky = W)   #发送按钮控件加载
button_cancel.grid(row = 0,column = 1,sticky = W) #取消按钮控件加载


tk.mainloop()

sock.close()

In [1]:
import socket 
import struct

# data of socket and file datapath 
ADDR = ('127.0.0.1',10086)
BUFSIZE = 1024
FILEINFO_SIZE=struct.calcsize('128s32sI8s')

# 接受文件
def Reveiver_File():

    recvSock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    recvSock.bind(ADDR)
    recvSock.listen(20)
    print(u"等待连接...\n")

    conn,addr = recvSock.accept()
    print(u"发送端已连接—> \n",addr)

    fhead = conn.recv(FILEINFO_SIZE)
    filename,temp1,filesize,temp2=struct.unpack('128s32sI8s',fhead)
    print('接收到的文件为：',filename.decode().strip('\00'))
    filename = 'new_'+filename.decode().strip('\00')
    fp = open(filename,'wb')
    restsize = filesize
    print(u"正在接收文件... \n",)

    while 1:
        if restsize > BUFSIZE:
            filedata = conn.recv(BUFSIZE)
        else:
            filedata = conn.recv(restsize)
        if not filedata: 
            break
        fp.write(filedata)
        restsize = restsize-len(filedata)
        if restsize == 0:
            break
    print(u"接收文件完毕\n")
    print('保存为：',filename)
    fp.close()
    conn.close()
    recvSock.close()
    print(u"连接已关闭...\n")

if __name__ == '__main__':
    Reveiver_File()


等待连接...

发送端已连接—> 
 ('127.0.0.1', 48478)
接收到的文件为： test.txt
正在接收文件... 

接收文件完毕

保存为： new_test.txt
连接已关闭...



In [ ]:
# python传输文件最重要的有两步:

# 1).将要传输的文件的信息发送过去,包括文件包,大小以及其它信息;

# 2).发送端读取文件内容并发送过去,接受端将缓存里面的内容写入文件.